Import Libraries

In [1]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App

-  create app.py file to make flask app
-  create Flask app to execute all of the endpoint you have made

2. 2 points - Created functionality to read or get specific data from the database

- create query to read data from database
- create function to execute read specific information into table from database

3. 4 points - Created functionality to input new data into each table for the databases

- create query to insert new data into stations and trips table
- create function to execute input data into stations and trips table

4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)

- create query to make analytical resultfrom the data
- create static endpoint to analyze the data from database, for example average trip durations
    
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)

- create query to make analytical resultfrom the data
- create dinamic endpoint to analyze the data from database, for example average trip durations for each bike_id

6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

- create input data for refering into query for post endpoint
- make query and aggregation function to implement the input

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [2]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [3]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059,Nash Hernandez/East @ RBJ South,3660,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054,Rosewood/Chicon,4055,11th/Salina,16


**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [4]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [5]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [6]:
# Get the data values
data = tuple(stations.iloc[1].fillna('').values)

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""

In [7]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (2500, 'Republic Square', 'closed', '425 W 4th Street', '', '', '', '', '', '', '', '', 9, '2021-01-04T12:00:00Z')



In [8]:
# Execute the query 
conn.execute(query)

In [9]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [10]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [11]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[6].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [12]:
result

'OK'

In [6]:
query = f"""SELECT * FROM stations"""
result = pd.read_sql_query(query, conn)
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2502,Barton Springs & Riverside,closed,400 Barton Springs Rd.,,,,,,,,moved to Pedernales/2nd. Original city ID: 16729,9,2021-01-04T12:00:00Z
1,3464,Pease Park,closed,1155 Kingsbury St,,,,,,,,,9,2021-01-04T12:00:00Z
2,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
3,2536,Waller & 6th St.,closed,602 Waller St.,,,,,,,,,3,2021-01-04T12:00:00Z
4,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,,,,,,,,,1,2021-01-04T12:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,4048,South Congress @ Bouldin Creek,active,799 South Congress Avenue,,32746,undetermined_parking,13,solar,40.0,5.0,,9,2021-01-04T12:00:00Z
94,3377,MoPac Pedestrian Bridge @ Veterans Drive,active,2105 Veterans Dr.,,16598,undetermined_parking,19,solar,55.0,5.0,parkland in ROW/eastment,9,2021-01-04T12:00:00Z
95,3794,Dean Keeton & Speedway,active,180 E. Dean Keeton St.,,32670,undetermined_parking,19,solar,55.0,5.0,,9,2021-01-04T12:00:00Z
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,,32671,undetermined_parking,19,solar,55.0,5.0,,9,2021-01-04T12:00:00Z


In [7]:
query = f"""SELECT * FROM trips WHERE bikeid = 248"""
result = pd.read_sql_query(query, conn)
result

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900294004,Local30,248,2015-10-04 15:12:57 UTC,1008,Nueces @ 3rd,1006,Zilker Park West,13
2,9900341244,Walk Up,248,2014-12-25 13:12:00 UTC,2823,Capital Metro HQ - East 5th at Broadway,2823,Capital Metro HQ - East 5th at Broadway,131
3,14047391,Local30,248,2017-03-15 15:28:32 UTC,2568,East 11th St. at Victory Grill,2568,East 11th St. at Victory Grill,15
4,11517679,Walk Up,248,2016-08-21 13:05:35 UTC,3291,11th & San Jacinto,3291,11th & San Jacinto,24
...,...,...,...,...,...,...,...,...,...
1681,11831110,Local365,248,2016-09-08 18:10:39 UTC,2552,3rd & West,2711,Barton Springs @ Kinney Ave,6
1682,10070270,Local365,248,2016-05-19 20:57:37 UTC,2552,3rd & West,2711,Barton Springs @ Kinney Ave,185
1683,8809667,Local365 ($80 plus tax),248,2016-02-18 19:06:56 UTC,2552,3rd & West,2711,Barton Springs @ Kinney Ave,6
1684,10789548,Walk Up,248,2016-07-04 23:17:55 UTC,2552,3rd & West,3292,East 4th & Chicon,13


In [8]:
query = f"""SELECT bikeid, avg(duration_minutes) AS AVG_MIN FROM trips GROUP BY bikeid ORDER BY AVG_MIN DESC"""
result = pd.read_sql_query(query, conn, index_col='bikeid')
result

,AVG_MIN
bikeid,
Block05,3385.000000
Block02,671.800000
19456,225.265957
17283,163.539877
172,113.118644
...,...
1475,19.400282
1386,18.772345
1797,17.613208


In [9]:
query = f"""SELECT bikeid, avg(duration_minutes) AS AVG_MIN FROM trips GROUP BY bikeid ORDER BY AVG_MIN DESC"""
result = pd.read_sql_query(query, conn, index_col='bikeid')
result

,AVG_MIN
bikeid,
Block05,3385.000000
Block02,671.800000
19456,225.265957
17283,163.539877
172,113.118644
...,...
1475,19.400282
1386,18.772345
1797,17.613208


**TASK: Create a Function to insert data for `trips` table**

In [17]:
# Your code here
def insert_into_trips(data, conn):
    query = f"""INSERT INTO trips values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'


In [18]:
# Example use of the function 
conn = make_connection()
data = tuple(trips.iloc[6].fillna('').values) # Randomly select a data
result = insert_into_trips(data, conn)

In [19]:
result

'OK'

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [10]:
# Get Specific Station ID  Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [11]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2500



In [12]:
# Run the query and get the result
result = pd.read_sql_query(query_station_id, conn)

In [13]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [14]:
def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

**TASK: Create a Function to read data from `trips` table**

In [15]:
# Your code here
def get_trip_id(trip_id, conn):
    query = f"""SELECT * FROM trips WHERE trip_id = {trip_id}"""
    result = pd.read_sql_query(query, conn)
    return result 
    
def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [17]:
# your code
@app.route('/trips/')
def route_all_trips():
    conn = make_connection()
    trips = get_all_trips(conn)
    return trips.to_json()
    
def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [18]:
url = 'http://127.0.0.1:5000/stations/'
res = requests.get(url)

In [19]:
res

<Response [200]>

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [20]:
pd.DataFrame(res.json())#.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2502,Barton Springs & Riverside,closed,400 Barton Springs Rd.,,,,,,,,moved to Pedernales/2nd. Original city ID: 16729,9,2021-01-04T12:00:00Z
1,3464,Pease Park,closed,1155 Kingsbury St,,,,,,,,,9,2021-01-04T12:00:00Z
2,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
3,2536,Waller & 6th St.,closed,602 Waller St.,,,,,,,,,3,2021-01-04T12:00:00Z
4,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,,,,,,,,,1,2021-01-04T12:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,4048,South Congress @ Bouldin Creek,active,799 South Congress Avenue,,32746,undetermined_parking,13,solar,40.0,5.0,,9,2021-01-04T12:00:00Z
94,3377,MoPac Pedestrian Bridge @ Veterans Drive,active,2105 Veterans Dr.,,16598,undetermined_parking,19,solar,55.0,5.0,parkland in ROW/eastment,9,2021-01-04T12:00:00Z
95,3794,Dean Keeton & Speedway,active,180 E. Dean Keeton St.,,32670,undetermined_parking,19,solar,55.0,5.0,,9,2021-01-04T12:00:00Z
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,,32671,undetermined_parking,19,solar,55.0,5.0,,9,2021-01-04T12:00:00Z


## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [21]:
station_id = 2546
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [22]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [23]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "my adress"
}

url = "http://127.0.0.1:5000/json"

In [24]:
res = requests.post(url, json=data)

In [25]:
res

<Response [200]>

In [26]:
res.text

'Hello Andi, your age is 24, and your address in my adress\n            '

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [27]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [28]:
data = stations.iloc[20].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [29]:
url = "http://127.0.0.1:5000/stations/add"
res = requests.post(url, json=data_json)

In [30]:
res

<Response [200]>

In [31]:
res.text

'Error'

In [83]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table

for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

98it [00:01, 61.13it/s]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [93]:
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, headers=headers)

17982it [01:57, 152.78it/s]


In [94]:
res

<Response [404]>

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint.
After created the endpoint, make sure to implement it to `app.py` file.

In [33]:
@app.route('/trips/average_duration')
def avg_all_bikes(conn, bikeid):
    conn = make_connection()
    avgs = get_avg_all_bikes(conn, bikeid)
    return avgs.to_json()
    
def get_avg_all_bikes(conn, bikeid):
    query = f"""SELECT bikeid, AVG(duration_minutes) AS AVG_MIN FROM trips WHERE bikeid = {bikeid} GROUP BY bikeid ORDER BY AVG_MIN DESC"""
    result = pd.read_sql_query(query, conn, index_col='bikeid')
    return result

In [34]:
avg_all_bikes(conn, 19456)

'{"AVG_MIN":{"19456":225.2659574468}}'

In [75]:
# Your Code Here 
@app.route('/trips/average_duration') 

SyntaxError: incomplete input (3792391672.py, line 2)

In [35]:
def get_bike_id(bikeid, conn):
    query = f"""SELECT * FROM trips WHERE bikeid = {bikeid}"""
    result = pd.read_sql_query(query, conn)
    return result 

In [36]:
get_bike_id(19456, conn)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,21610363,Local365,19456,2020-02-24 16:32:42 UTC,3635,13th/San Antonio,2561,12th/San Jacinto @ State Capitol Visitors Garage,6
1,21613188,Local365,19456,2020-02-25 15:07:47 UTC,3635,13th/San Antonio,2561,12th/San Jacinto @ State Capitol Visitors Garage,4
2,21619370,Local365,19456,2020-02-27 16:17:07 UTC,3635,13th/San Antonio,2561,12th/San Jacinto @ State Capitol Visitors Garage,9
3,21789006,Local30,19456,2020-04-14 14:39:13 UTC,4062,Lakeshore/Pleasant Valley,4062,Lakeshore/Pleasant Valley,38
4,21789129,Local30,19456,2020-04-14 15:18:19 UTC,4062,Lakeshore/Pleasant Valley,4059,Nash Hernandez/East @ RBJ South,83
...,...,...,...,...,...,...,...,...,...
89,21751064,Explorer,19456,2020-04-01 17:07:17 UTC,3838,26th/Nueces,2547,21st/Guadalupe,35
90,21728832,Local30,19456,2020-03-26 14:53:21 UTC,3838,26th/Nueces,3793,28th/Rio Grande,52
91,21728658,Local30,19456,2020-03-26 14:09:41 UTC,3838,26th/Nueces,3838,26th/Nueces,43
92,21728493,Local30,19456,2020-03-26 13:26:53 UTC,3838,26th/Nueces,3838,26th/Nueces,43


## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

In [ ]:
# Your Code Here
@app.route('/trips/average_duration/<bike_id>')

In [37]:
def avg_bike_id(bikeid):
    conn = make_connection()
    avg = get_bike_id(bikeid, conn)
    return avg.to_json()

In [38]:
avg_bike_id(19456)

'{"id":{"0":21610363,"1":21613188,"2":21619370,"3":21789006,"4":21789129,"5":21808545,"6":21807372,"7":21787332,"8":21786847,"9":21928479,"10":22007400,"11":22015662,"12":21866256,"13":22165726,"14":21617944,"15":22090049,"16":21957130,"17":21928862,"18":22025684,"19":21929675,"20":21884637,"21":22115449,"22":22161759,"23":21611430,"24":21614681,"25":21621988,"26":21888508,"27":21705407,"28":21712910,"29":21958272,"30":21943932,"31":22077522,"32":22076766,"33":22078731,"34":22080393,"35":21888038,"36":22080986,"37":21926869,"38":21901776,"39":21919861,"40":21927770,"41":21929379,"42":22129846,"43":21728207,"44":21928202,"45":22099622,"46":22133022,"47":21979145,"48":21653847,"49":21608849,"50":21609117,"51":21608697,"52":21608397,"53":22136585,"54":22136216,"55":22027877,"56":21936347,"57":22162380,"58":21728308,"59":21650495,"60":21600164,"61":21610149,"62":21618684,"63":21963502,"64":21607993,"65":22090026,"66":21705944,"67":21706131,"68":22091505,"69":21969940,"70":21783894,"71":216

## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

After created the endpoint, make sure to implement it to app.py file.

In [44]:
def dictionary():
    input_data = request.get_json(force=True) # Get the input as dictionary
    
    print(input_data)
    specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")
    # Subset the data with query 
    conn = make_connection()
    query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
    selected_data = pd.read_sql_query(query, conn)

    # Make the aggregate
    result = selected_data.groupby('start_station_id').agg({
        'bikeid' : 'count', 
        'duration_minutes' : 'mean'
    })


    # Return the result
    return result.to_json()

In [53]:
data = {
    "period" : "2020-08"
}

url = "http://127.0.0.1:5000/dictionary"



In [54]:
res = requests.post(url, json=data)

In [55]:
res

<Response [200]>

In [56]:
res.text

'{"bikeid":{"2494":244,"2495":122,"2496":74,"2497":131,"2498":142,"2499":170,"2501":280,"2503":79,"2504":71,"2537":61,"2539":203,"2542":111,"2544":105,"2547":175,"2548":108,"2549":198,"2552":264,"2561":104,"2562":28,"2563":201,"2565":42,"2566":247,"2567":173,"2568":26,"2569":25,"2570":73,"2571":49,"2572":137,"2574":244,"2575":387,"2707":427,"2711":160,"2822":23,"2823":25,"3291":105,"3292":57,"3293":77,"3294":57,"3377":239,"3390":127,"3455":127,"3513":181,"3619":172,"3621":244,"3635":58,"3660":124,"3684":57,"3685":114,"3686":251,"3687":209,"3790":69,"3791":101,"3792":105,"3793":189,"3794":83,"3795":88,"3797":86,"3798":175,"3799":119,"3838":106,"3841":158,"4047":42,"4048":107,"4050":132,"4051":13,"4052":26,"4054":27,"4055":21,"4057":97,"4058":65,"4059":227,"4060":84,"4061":358,"4062":257},"duration_minutes":{"2494":52.9918032787,"2495":28.0901639344,"2496":47.8918918919,"2497":39.3893129771,"2498":23.1408450704,"2499":56.6941176471,"2501":24.1678571429,"2503":53.8734177215,"2504":30.6056

In [62]:
query = f"""SELECT * FROM stations where station_id = 2494"""
result = pd.read_sql_query(query, conn)
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2494,2nd & Congress,active,151 E. 2nd St,,16733,paid_parking,15,solar,45.0,5.0,,9,2021-01-04T12:00:00Z


# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. Make sure you have implemented all the endpoints to `app.py` (including your custom static, dynamic and post endpoints).
3. Submit your `Bikeshare API.ipynb` and `app.py` file in your github repository. The dataset is optional to post since it has big size to post. 

